This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_best_worst_per_point(y_true, y_pred_list, savepath, metrics_dict,
                              avg_stats, title='best worst per point', label='target_value'):
    """
    Method to create a parity plot (predicted vs. true values) of the set of best and worst CV scores for each

    individual data point.

    Args:

        y_true: (numpy array), array of true y data

        y_pred_list: (list), list of numpy arrays containing predicted y data for each CV split

        savepath: (str), path to save plots to

        metrics_dict: (dict), dict of scikit-learn metric objects to calculate score of predicted vs. true values

        avg_stats: (dict), dict of calculated average metrics over all CV splits

        title: (str), title of the best_worst_per_point plot

        label: (str), label used for axis labeling

    Returns:

        None

    """

    worsts = []
    bests = []
    new_y_true = []
    for yt, y_pred in zip(y_true, y_pred_list):
        if len(y_pred) == 0 or np.nan in y_pred_list or yt == np.nan:
            continue
        worsts.append(max(y_pred, key=lambda yp: abs(yp-yt)))
        bests.append( min(y_pred, key=lambda yp: abs(yp-yt)))
        new_y_true.append(yt)

    worst_stats = OrderedDict([('Worst combined:', None)])
    best_stats = OrderedDict([('Best combined:', None)])
    for name, (_, func) in metrics_dict.items():
        worst_stats[name] = func(new_y_true, worsts)
        best_stats[name] = func(new_y_true, bests)

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 15.5/24 #mmm yum
    fig, ax = make_fig_ax(x_align=x_align)

    # gather max and min
    #all_vals = [val for val in worsts+bests if val is not None]
    max1 = max(y_true)
    min1 = min(y_true)

    # draw dashed horizontal line
    ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

    # set axis labels
    ax.set_xlabel('True '+label, fontsize=16)
    ax.set_ylabel('Predicted '+label, fontsize=16)

    # set tick labels
    #maxx = max((max(bests), max(worsts), max(new_y_true)))
    #minn = min((min(bests), min(worsts), min(new_y_true)))
    #maxx, minn = recursive_max_and_min([bests, worsts, new_y_true])
    maxx = round(float(max1), rounder(max1-min1))
    minn = round(float(min1), rounder(max1-min1))
    _set_tick_labels(ax, maxx, minn)

    ax.scatter(new_y_true, bests,  c='red',  alpha=0.7, label='best test',
               edgecolor='darkred',  zorder=2, s=100)
    ax.scatter(new_y_true, worsts, c='blue', alpha=0.7, label='worst test',
               edgecolor='darkblue', zorder=3, s=60)
    ax.legend(loc='lower right', fontsize=12)

    plot_stats(fig, avg_stats, x_align=x_align, y_align=0.51, fontsize=10)
    plot_stats(fig, worst_stats, x_align=x_align, y_align=0.73, fontsize=10)
    plot_stats(fig, best_stats, x_align=x_align, y_align=0.95, fontsize=10)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ])
y_pred_list = [[226.42799331445633, 226.79025512808263], [161.50885000121048, 158.73105761111444], [135.6634993684213, 140.14439003400085], [278.7091828147335, 263.5175315734357], [296.0550607588735, 303.1597482235936], [196.7467282750428, 193.16509115483217], [205.72242207928565, 206.4968499381456], [184.64369286967138, 179.14267844099714], [204.81330652116432, 205.43537129234775], [190.31006536567864, 194.6926524848455], [225.96890127341973, 226.46757627669507], [168.69624487961488, 182.57192960313893], [168.74102132204575, 168.58825629257095], [179.96343225507155, 170.10410517196078], [162.93271385413308, 158.75301351062365], [168.75352984671298, 161.8637424252646], [173.21791924473123, 170.72780538291], [154.0688739892238, 155.87228744685274], [156.40442595567836, 146.56386979784912], [158.85269888910392, 158.66048723871938], [151.35561536407573, 147.8309897220435], [151.96269007247338, 156.85972209987915], [269.4979707104651, 274.03182692443573], [261.4117481054401, 267.16246663825945], [279.7741649388065, 277.6913529063544], [244.87450993741908, 238.42478028679912], [273.7322190039218, 281.6229676202429], [273.77701945968954, 263.0196080801279], [225.72008396290082, 218.39856712138885], [316.34127477493774, 330.3395889289024], [245.56103058058778, 253.21828214586276], [212.1031094560388, 218.39789951493177], [138.3551119176102, 151.19630400997085], [106.03443242178966, 103.05351398889452], [106.04779737444943, 99.27950075119], [205.59192851007703, 203.32514302706375], [180.09772761080984, 172.76463989306276], [157.1716442525619, 164.48741908575028], [120.83832980832305, 116.53383181025787], [154.74652440964772, 162.18229174599531], [112.73210861326866, 123.30184628295794], [113.14998817525175, 116.94505580820179], [132.73255735760875, 129.03425825886444], [118.06456946723293, 114.46589865512304], [93.51457708298368, 95.94294354408382], [121.01464370265532, 121.40143902710886], [86.78179530641202, 92.95332445537969], [90.43903759296677, 89.22517005769491], [113.88490364568315, 105.98418212379596], [84.68087005142823, 86.75926374945571], [93.16264277166096, 92.26382557297491], [95.05267213263416, 90.9991402401545], [94.52251001791007, 87.91044174749398], [91.97274348034438, 92.29992646311126], [247.63465253889683, 230.16707276123935], [147.1691186538282, 152.54862432606373], [156.56906304601887, 153.35381048608417], [147.31438956390664, 151.10934563885795], [159.547793282128, 152.43948699179998], [149.94510527921653, 153.4250137169734], [211.59708203046853, 209.719611765662], [167.5265010198592, 167.381232548666], [137.295265520476, 136.44011350017044], [284.15857110647886, 286.8651354974622], [169.22312223425027, 178.49249620244407], [123.60169394628974, 129.02726523558854], [94.78927581986315, 102.1837506904578], [209.1766511581219, 196.73304703185465], [182.21724200674132, 176.66815737098452], [136.75464259144712, 138.86033742707602], [157.14853886749734, 158.347120232506], [141.6941191286628, 138.8255282444007], [152.7242898851787, 155.86459486402487], [151.6613323756455, 145.48935983151483], [164.64219610905607, 160.41920319523308], [123.72796365467764, 122.37198684445293], [120.01297744636099, 117.13982643903645], [135.85277897027134, 141.1357430297445], [100.35045099001192, 97.29303587394658], [102.24978024994876, 99.4873399260229], [126.53731467335922, 134.7314669921245], [97.92808150003611, 98.44915876471843], [92.220255780136, 92.88510268934354], [111.45139881885895, 108.87779868200933], [87.63702803077979, 88.41137798627744], [85.76788936771678, 85.87367387339944], [232.79564153901427, 232.89962406065894], [117.23206908738865, 125.38895573033341], [170.46745618274173, 173.56270230866272], [120.91630736984193, 118.32517100388759], [171.22970082189093, 171.2092092665146], [179.500119565177, 169.55253382002107], [200.1447839600324, 201.9067539397691], [182.07011168974557, 174.04235382997683], [159.56996155618657, 167.55860788896925], [163.35071980879388, 158.6903751686191], [226.9802923447093, 231.93788765358846], [169.58072513894166, 165.6320991870046], [159.0982653609334, 151.58824486490568], [290.37424028005705, 291.9224395752348], [291.43658628266803, 288.3187944628347], [169.35211446824601, 185.59546570119625], [191.34788723484579, 195.1572400098152], [167.0243959752731, 173.73351452265695], [193.9748809826051, 186.07108665403373], [189.68955704461666, 192.37144047984305], [217.06005266390622, 212.51400078698643], [175.10600402678676, 165.44283908720763], [151.62354035676464, 159.64261747083015], [173.70099935353346, 170.82231881484864], [159.9580843249332, 161.7311060424726], [150.9142143554842, 158.58269298772473], [169.73945273703595, 173.85061464035485], [150.02669319366453, 146.5144940328562], [151.6635883552443, 151.35620487002626], [156.6994713458378, 151.40414181928764], [157.82656971337056, 153.43882737477816], [143.66045608943963, 152.63567032042587], [316.1019825956674, 307.57002145620055], [242.10404226487415, 232.23549031970452], [267.498595046788, 265.68403886753845], [212.68710842947414, 230.51979870910856], [263.11162852102865, 261.79489409709504], [256.5142606493076, 258.3975023045936], [316.26351344978593, 302.76124064088486], [237.50084399251193, 235.1604900792688], [193.74227662733887, 198.011096491995], [301.55177273614447, 304.32803263429514], [142.08040135983939, 132.55957400549084], [101.1325446920647, 102.18368516211913], [102.52850187215199, 96.44982194321541], [196.5657990731938, 191.94626734805368], [175.28393585242586, 171.85477303896582], [151.77524263610414, 159.07233436713958], [118.4717855552517, 124.05542477034477], [156.76402782377323, 151.0246806181795], [106.74235831608273, 108.99590954448686], [107.56820259106371, 106.95679873986268], [127.87785938848569, 131.81404893150562], [113.40030126445534, 112.957409274513], [106.27314237326928, 102.18574652790537], [124.5428725789356, 124.9008488643777], [90.76320044186161, 89.52929780098756], [91.28230176833799, 84.84218675484311], [109.36253063355022, 108.09648034208202], [70.23489659757128, 71.62344191089316], [85.41965510147314, 88.51647909448599], [86.17192524024924, 84.41075403405726], [92.16181891411028, 92.40284406085672], [89.89659335879074, 87.95519863021791], [215.9361920494581, 225.24416827469724], [148.80561092764046, 163.4987529691274], [151.58512524845352, 162.9842124416438], [160.70867031357966, 148.14462670449888], [147.0807779347975, 158.83155705864476], [148.44783445668702, 144.0990397406788], [196.02173355332886, 194.6151316296036], [169.06862744625795, 159.7753226996137], [139.97844016308173, 130.2516386352514], [294.0532994531753, 287.6008489174019], [28.327048649803693, 26.608278809765324], [37.263361575886066, 37.46322477990331], [75.217682651333, 69.90713065875171], [88.78013868126962, 91.80869187648142], [42.74395500442725, 52.926726619852445], [28.2379050887831, 37.569052586280065], [103.70549263073083, 129.02024201305218], [208.01640619783174, 206.32277223399643], [91.63903771126516, 94.20159052760692], [45.544249141233834, 50.778896531613746], [35.663953318247195, 36.650039269466774], [28.66219185835311, 20.21230356590638], [33.25723757893346, 18.288794941408646], [69.56141535359404, 68.42762796560498], [64.58725312647067, 56.61606455373948], [54.89011085289482, 46.35239070137766], [52.39642076795009, 50.29271572208617], [19.56002790813874, 14.036254806014753], [105.4148322799496, 114.12082026406463], [178.48330622567906, 152.1048192456319], [117.5300847772998, 117.58502520469163], [129.0226614568083, 130.66210026066545], [129.88511790422825, 131.50898587394198], [137.19894468320462, 132.56516046564047], [127.69659011772367, 127.10724358469301], [85.16412912005978, 79.37392428096317], [66.35870287530702, 61.73562826254283], [279.0324935034052, 267.5253581809362], [80.3285954845644, 80.19619020756016], [96.74102958606284, 80.20842781581467], [85.76819945195581, 100.15192914182228], [147.95730472078623, 140.4571803294532], [165.60282508905786, 166.6473800639879], [84.60605246040234, 71.6379682546962], [74.98685678460055, 77.10585996328075], [107.17232928950492, 104.52374471323745], [105.04500877654382, 86.97934705911484], [247.2182041418294, 245.09108694970297], [80.32993540567429, 81.98963388758955], [93.38565581034136, 85.60712705105823], [87.6431044759508, 111.72232711079039], [193.10182142070974, 179.76682764257194], [93.00686657532947, 87.93771902934841], [106.16623310595264, 104.5578594019052], [106.71235837856864, 105.97548110492824], [117.56184545641048, 118.66388995015592], [77.18744515245953, 77.0314632439508], [91.38851833499189, 91.16166316068512], [95.99202100311845, 88.45914303584681], [98.35359272546143, 99.7356036310888], [85.34219614575069, 85.76841636132306], [515.5869902125357, 582.2734291132832], [179.82476903857176, 170.23072418137133], [241.46668220519047, 226.147365523148], [405.3818767939126, 374.9033674614982], [509.945373522402, 508.58751029045357], [78.91100801442677, 66.24403360507108], [85.81707137302877, 79.19116092499075], [65.12920629962724, 62.76836454936955], [71.58965320686548, 69.02323592305595], [75.71085801735471, 76.5220369830746], [81.51874048447849, 82.2503057248934], [66.23501203779284, 64.1625145831652], [100.1722583128999, 93.97732984027635], [73.93805580918779, 78.48045124831751], [68.87214254244032, 66.92280288184345], [94.77187269978864, 93.6687626281205], [83.42283580180319, 82.25736294356119], [100.9824910243533, 97.03023372436124], [129.0510577299878, 122.94070845049384], [125.28366659171442, 120.07982737413838], [110.55449677185199, 117.66783823176274], [108.34441892475738, 107.21558544021025], [132.22506962260104, 133.2277436227268], [131.80181343599077, 135.82036316718066], [140.84422390861937, 146.22261985690503], [151.90362111141292, 150.3845263944768], [120.10525419640452, 129.45144943703593], [113.33641583185852, 110.99718570308619], [283.5382715135609, 291.43522467676615], [135.43465089920534, 133.6264826596173], [145.71266252951403, 144.76687037704298], [179.21773533167413, 170.0490526212401], [234.4240068720021, 224.77396363078196], [172.07734919511108, 171.16707529615206], [150.47503771378325, 155.19900445709436], [164.57514849755108, 166.04081064424025], [180.61051281829415, 177.88577333382148], [176.15948146196087, 171.60779655200952], [8.180642424484496, 3.426370550249084], [69.69978351481785, 69.57865429173353], [78.36320959058872, 73.11049041692212], [24.019367077996748, 22.45817694288661], [71.6372038530798, 54.62278088972299], [109.89773518138787, 105.92192189067285], [115.5990967828388, 108.09617672650265], [123.72715250608368, 109.6850124355276], [311.98386995528193, 297.89734509600737], [120.43491347853373, 124.26812988013828], [126.72814780916917, 123.43472306628811], [179.30023579738582, 181.4232227516346], [17.67376080296448, 20.82285374762023], [99.01399074284944, 90.45832701485762], [96.97670639356659, 82.88713522133177], [85.02041234497916, 94.95510167559374], [83.00955822306719, 83.23980146121418], [112.2981322190152, 102.51437705155183], [104.29525226569531, 132.40682009367427], [217.74890485911428, 220.98199057687773], [142.78507367775867, 132.58582522080604], [131.9855264195878, 139.97024343907938], [153.7942305704554, 146.32466218795292], [82.85331018450113, 65.05474937359027], [202.455234718193, 196.31533890332696], [195.92586257323515, 195.8106302137813], [141.46333416256974, 158.5873388027438], [115.08481155408886, 111.91914022149676], [78.41818957744354, 73.3577104865766], [98.566922622033, 96.61146340718548], [118.24997693536776, 117.36867308441713], [203.75306913635254, 199.73862415022518], [146.73239441638475, 155.63553688449082], [148.89855849279365, 143.34440232413766], [146.2128773725778, 150.8453833716797], [143.2711568675385, 139.23154794311387], [112.79522201298295, 116.91558540533015], [112.52652005142232, 106.60021969394731], [110.71531597580542, 110.45296289999209], [36.271901380711114, 39.754267069034285], [70.86697496351843, 62.36041170580552], [77.06134966596606, 79.79461505804761], [17.275007657311946, 20.50319223299411], [106.32978154053102, 110.43660193400136], [91.75563078624609, 91.1896580486607], [79.73485999259913, 87.89002766235511], [94.35238216452788, 83.67042407652475], [138.75914405062554, 130.55919576881735], [117.35883154381372, 113.12073980934366], [116.11025990221611, 118.08459586686115], [118.13431362684605, 112.20356349105352], [286.19499128069526, 286.6943314223357], [143.2901173009297, 147.5852642995097], [155.59845755352882, 155.54397655649115], [201.18202709928067, 198.50690140914637], [245.39865973883883, 245.17912811249192], [18.779984463701204, 18.003065406254812], [103.2837369814232, 93.97830000273854], [103.310371399265, 94.65908336860946], [84.5186988200523, 83.09728635506804], [70.33272360991377, 68.83841071574827], [41.286653108322895, 34.002590635018805], [26.461245253511336, 27.474319045955607], [45.951542224555794, 39.246995299692855], [60.96468024244335, 48.865227303089625], [112.80433332406744, 109.14811243637155], [25.766231242849926, 26.418157736921067], [51.34403235970373, 66.58057175521137], [161.89579947827139, 156.5552593082837], [60.71402100475942, 58.62032065204244], [42.52930589828196, 39.09774856445185], [8.63343537820191, 15.177683765204947], [37.85373630243025, 34.11182521216227], [30.845788914530615, 40.89678488321971], [116.64413158889155, 121.08275119213673], [29.32646726541391, 30.512546643466827], [35.28319280764907, 34.53335042579975], [40.47221141784193, 41.85347416059878], [46.26322282372546, 51.0659721524864], [78.30992208612042, 73.76786132338538], [159.4616486494245, 160.89705661209985], [61.18015347549027, 59.03982952470068], [21.437947515805824, 34.30598739231502], [1.7657580797007455, -0.867932125437715], [53.843603151857394, 42.638723734164444], [30.30954453097077, 28.034373217830158], [53.82094699146802, 63.77593930667237], [146.2489521488455, 136.93756283743653], [78.36695872538267, 77.40987569807203], [56.209201595005126, 51.40579129405571], [65.31785765704353, 95.80665604603332], [134.0193181448917, 145.79360418337563], [66.38636683032479, 71.74007039295088], [46.65471020392124, 48.29629328734501], [100.49245421663257, 77.58888259080246], [122.3529941043257, 104.84188090548226], [200.3098062353456, 153.4352465012682], [138.51200476653574, 146.52306749442243], [59.358515426265996, 55.6549205645899], [45.09989088795761, 41.52910941423204], [144.25703801117257, 139.56123150423954], [123.70872186919391, 121.46568962445247], [134.02664231244975, 129.8900755320007], [153.27105028472306, 154.74910329846395], [109.41889225169174, 120.51630807175735], [281.91030226760745, 286.4580511709196], [139.67293224251995, 137.2531675060409], [135.35410926664454, 147.31097865302985], [164.15363414008817, 166.2149922015471], [220.82244079240354, 222.33551817014364], [171.5655470106442, 169.98799118553018], [150.0062194278977, 155.37367048380588], [168.30226277866046, 154.98837981106388], [168.2218547018804, 164.89933105514342], [168.62851970191912, 165.99551126858358], [285.93206834474984, 284.09937483542524], [142.55681887185185, 142.91337938163582], [153.4316764045675, 161.67480749652265], [201.7379528466357, 196.68032671886175], [250.069771830925, 243.79055081185516], [75.59603770029145, 78.42831420171177], [96.02719438578939, 98.33966764612259], [75.26612832305364, 73.90187377291358], [84.94872689038135, 83.42327901855668], [52.9181998182306, 54.639590440213965], [78.65591276414283, 86.35683293344988], [-0.7939990171780096, 0.8242524231719415], [6.140734988509799, 15.01269224132399], [22.755224220496615, 24.541034586875526], [112.73566953940258, 107.21935250503248], [49.243477564043054, 45.48527062300269], [49.33372340038741, 49.07143539419516], [44.645863262657656, 44.985722355354724], [61.14162029461633, 49.34771093380499], [76.18337432507869, 72.35968731337641], [84.7081341347199, 85.19489447202847], [55.705779554784016, 58.230457947760556], [55.74948175066632, 54.56430484547775], [59.20102591516179, 51.80663601969226], [52.858389906773766, 56.68113726569254], [12.265814881408744, 13.038048653919674], [25.35321964116997, 33.6097259354533], [57.60465807437788, 58.73229151666696], [54.04363336274284, 56.480813318423344], [50.38611858782792, 53.73486548114295], [68.29157035579081, 77.69235667161769], [73.4470685174561, 75.80401350058753], [82.07971081395105, 80.63143423558975], [59.89210154892697, 60.79129568732884], [33.95307636466856, 36.82531966590071], [60.787670776233654, 66.16105058089477], [31.21855204644183, 32.588587371828595], [48.01727625803949, 49.20535127131444], [8.927137459436487, 7.58912851044623], [7.228571101856346, 5.8080656333925305], [-1.0436732016909076, 2.4838634163365754], [-3.403580940617161, -2.7795996121170248], [9.999312902135571, 9.715339168500094], [13.699589751603071, 14.438603056822913], [6.799345819010767, 6.0705928734348475], [7.4795472750124645, 9.037664441965944], [-2.004823327448534, -0.4484902953810206], [32.740158595489504, 33.376749874177136], [50.366479811329384, 45.164402915426976], [45.98934698453462, 46.85659655061323], [36.47354698909828, 47.11483695103453], [41.48484084692171, 38.05709181451658], [37.409101759838386, 31.68750544393569], [32.96139773868963, 27.641805056327257], [30.90449742013007, 32.5093791275578], [35.128660331398436, 35.73268210631255], [29.923458047675652, 26.996104072741474], [21.67023573757291, 21.901798850506943], [24.010264361798463, 18.79066323718729], [24.354121172470485, 22.24594309012577], [74.73585430652503, 67.06810718924811], [52.184114600385215, 51.57428012473335], [59.76396725191597, 57.925877742362886], [22.50376670532114, 21.35368217201534], [48.61572321495261, 54.81972012419942], [64.26400544131772, 68.6027460700716], [51.88369864717117, 46.125737573965495], [61.51704502466478, 64.69717582559676], [19.885729042285035, 20.203152989656814], [32.99199222252979, 25.91173881542329], [0.05871021819494615, -4.565830364185153], [-1.4482633874088862, 1.3458786414618373], [23.190605607795867, 24.838269964285317], [35.05694041209762, 39.16781904562032], [16.85896569828023, 14.16368505778155], [19.15743817185455, 18.641906760820717], [15.644512074650066, 14.809653493048302], [23.606661397185604, 26.25422490941301], [52.19842992224483, 50.37709562061525], [69.95098501959922, 70.05011383681438], [24.552633598356678, 25.005541523490535], [28.023956468694486, 27.901986003286794], [39.41992280935247, 40.381406664299334], [18.199562985120394, 19.56088370709162], [24.0011724411778, 26.11644416089638], [17.818335631722107, 20.601256034451865], [25.525412539899648, 22.69657626164107], [41.68052855742019, 35.22040969506061], [56.204895130102955, 53.791996278474215], [21.8844544228592, 19.302262781349214], [26.18359774114947, 28.426161257436632], [21.774999562518452, 21.5920935133187], [42.52723144880343, 40.81731485738891], [46.896187793964486, 45.92669900463794], [58.44761308940113, 50.48528165064644], [18.41592662418566, 20.10442354289276], [92.42156665052687, 91.2957885591867], [145.9031753202386, 145.81552267286878], [82.62484820520672, 82.35884361170156], [134.99659179615557, 133.18738784732307], [42.572936533048534, 48.81601248439005], [83.77288798686085, 90.61492331627241], [52.94578069837496, 50.28664575883752], [77.58360130073272, 74.56898028074713], [50.504587081980276, 52.58832728075039], [84.01530806668956, 92.89953007077462], [33.01775329631653, 35.2462800462274], [39.72195580998661, 42.950855020785326], [32.32589476054586, 27.97171378032686], [52.05230798168249, 54.39201452967094], [83.9898960043117, 83.80510618408083], [136.03054914757962, 134.0998136393227], [44.80201508179663, 46.85979536983084], [65.01431996384929, 64.03665060237293], [93.8100970863635, 83.47124681300551], [31.16171197829867, 26.802365365878497], [36.27282904093658, 34.117336101187604], [53.51223248125095, 40.30835638906832], [36.43905526107147, 36.275049582946124], [47.57879250446038, 44.739188542418255], [68.59712270752324, 63.06485226249368], [29.626584917625216, 34.67567983251791], [58.590566351666595, 42.37684740923915], [69.13454668699697, 59.03486007212873], [42.61307359530349, 38.48226558828235], [44.583368694717606, 37.602335853064254], [57.65881357187517, 56.059582183696534], [87.93470034412687, 85.4660146308685], [125.99225020731812, 124.29154441589141], [30.922152841492466, 29.927773973695878], [51.28778015291711, 49.76961113299251], [47.46289978478191, 46.30465326876447], [59.785397084434805, 67.06180231394734], [35.588184965192795, 37.432932518076115], [42.63184018338633, 48.02648885609893], [-5.706092668630205, -6.544576151759898], [-2.2470318534947182, 0.14021103235664611], [38.21242345408844, 39.11272430165431], [46.91461234646446, 46.24090199759715], [32.14779262874862, 30.912840252598755], [34.99938978605741, 36.19266191954638], [31.278281827194206, 32.86196558263086], [40.9672503219749, 42.49493342439999], [50.64682840377955, 47.69888214990161], [62.97170540428607, 63.26018933909516], [47.223389010602865, 47.73893488247904], [40.136926344784854, 43.24451285621166], [46.12991160626996, 44.74526610473367], [30.931378181872304, 29.577972827584006], [29.870197307178465, 28.67211643774496], [32.36726193211133, 29.765256589784173], [28.56764464776927, 30.03330509347873], [17.940328940550216, 19.48692459657741], [28.84582844360048, 26.125849101980037], [27.822119426913048, 29.101507607315288], [73.10895542624579, 57.194974024345726], [70.19879074928978, 68.81283153820169], [64.38292421091626, 67.13656507624842], [32.442981315439454, 31.679637181113378], [178.06495272350531, 181.39679715594207], [247.86337220150116, 261.7035368046939], [170.91768312776188, 172.38587855128407], [232.50348525789718, 228.49098113338925], [108.99027199298746, 104.07969232660791], [111.28575071785845, 110.59338124737806], [13.536297123735917, 9.028892128063205], [109.55614985330908, 115.1490117256647], [115.89334435482105, 127.11582023741852], [106.05488929749902, 107.38871310483745], [106.20159192914764, 107.42699991561676], [103.80207785805989, 104.13866880207837], [106.43929438021956, 112.91690750030017], [167.4313937366262, 170.86933768518827], [235.54819672160772, 234.85006376801917], [101.9362263076107, 98.57946511175601], [91.23774832213388, 88.44983100229393], [96.11338848518903, 90.18896850528006], [86.38644646364088, 88.0263629477306], [95.42697274870517, 92.9150485773948], [87.26376865400468, 80.94853797401649], [20.01984235177801, 13.637007633514344], [45.9300504589999, 36.18188779582425], [77.95830110652528, 57.39329418331178], [90.87471483408211, 96.57554631482068], [84.06687977712829, 78.81092427130505], [110.98978068043775, 114.46049098338548], [52.5839535308548, 51.00081522648853], [86.6022071061896, 86.80751650469753], [42.13245580200999, 40.648476927154434], [79.61955638627121, 81.63485161517093], [14.426866188713326, 13.818703994573388], [25.202509661733647, 30.716530003622644], [21.05799589915055, 19.978424337458055], [30.593541557491132, 27.105387827209462], [17.869454466720747, 18.31266611372162], [29.644765963676722, 34.14431459483524], [14.806197430251231, 21.39442368984741], [17.67144638061533, 21.827243421169925], [13.79390210842671, 19.997070506962224], [16.841813135960493, 14.233773632494017], [48.64018829711392, 44.813245388751646], [80.08766472833413, 72.02409733199804], [24.281718661314855, 27.874055685598883], [60.40595160318878, 60.6867419266713], [66.2631008965389, 62.47286869044441], [9.033276870184242, 2.8064308089991243], [16.232282512902664, 18.58336344359293], [32.83217605356349, 32.72807945483804], [15.74638193489475, 19.69815342982865], [35.62888598039828, 38.67276162261832], [46.37661309230454, 47.75111875786537], [35.40829990165635, 34.2366036061414], [49.89622834723132, 50.50769649301697], [93.99871928831634, 95.53101777537921], [11.089967320461568, 13.337968744621406], [23.522742771517095, 20.094272387082555], [35.05244685637132, 35.027258988038795], [35.60360848924357, 31.672874098827634], [78.94173076947098, 82.90432587353285], [74.65980416814381, 56.53268556005122], [14.86415845989365, 21.12556509318491], [28.09350722197921, 28.691831866225712], [34.58076631129242, 34.036864943359234], [25.00366644434927, 24.967426583230996], [25.94408441968808, 26.64919212964446], [34.15610748957458, 38.05215377369863], [44.188641427614634, 52.28358203067993], [31.07102018485974, 29.156102263160218], [44.827343832892986, 46.18669303421777], [36.05927582705921, 35.33180945879711], [51.459994715227936, 50.27907425244856], [26.46900279906717, 24.11538712472033], [30.658327153798226, 28.879333302156073], [27.41105647424929, 28.297397854881147], [38.69711085629841, 35.790718620838724], [25.942016600777137, 26.124044466652958], [26.82041952885718, 27.508369207751265], [61.42905056517495, 63.725262585045044], [79.09424871992337, 75.74368688487944], [55.69823497801519, 60.273198181220835], [31.182470989294913, 31.372133300997824], [27.984389888620427, 30.976550022445522], [32.77781787452927, 32.31889472836326], [50.05735672374533, 47.987121635692596], [48.14529982272656, 57.16610276941094], [47.62718344410943, 48.83777040783389], [22.988514939618522, 24.780379006231797], [21.728394756683294, 22.894583682831257], [25.045329513613666, 21.981899945411705], [32.16042543230741, 29.666744421761166], [35.09562667622698, 32.68179329103545], [30.451460700477753, 31.963845515000926], [74.78718350451761, 76.65611581293706], [82.07122667305802, 73.3797127729502], [57.63252625200147, 57.29407746966871], [23.710268290436765, 23.54008668878788], [14.598327649937445, 12.001710805828006], [28.812456122527806, 29.83708280039555], [11.919998075154187, 10.852900004397164], [22.918900111678404, 19.74430723058751], [17.990808728058767, 19.22675199317743], [35.70503656371245, 29.91913280351322], [13.157839933314989, 17.234225476626293], [25.504342808635677, 23.470245530461277], [24.324444829104078, 25.9808645612343], [29.95462850448218, 28.731611325615596], [4.059914206416296, 9.929749090682908], [8.498100181423183, 9.786378884348416], [25.44973272362553, 27.549257960410678], [10.928072460474155, 10.809980468117633], [22.445988840018536, 20.74951686093153], [57.732308437155496, 56.128446009427364], [50.55809245130342, 47.99912335911799], [45.517187640700115, 39.51763358472431], [30.576671518479607, 25.14023401518196], [29.29954319125065, 24.038741077087053], [27.221500797793652, 26.900368261640033], [50.264961875098436, 47.52148119331565], [40.70951421655812, 36.31107114580334], [36.27988251959886, 28.893574600712384], [-2.7502858863982738, 3.719469860586056], [1.4593423561119536, -2.1612805918150855], [-3.284242652382673, -2.2515795910503726], [29.425579097981938, 29.300943987414996], [21.22334072430408, 23.95016826981606], [20.828667659133345, 24.60657520748083], [63.57432379602295, 64.57024255574777], [55.0338142186406, 56.060363050816164], [59.83720904859203, 56.26384584923589], [5.318718378898183, 6.2877691634809025], [27.50271629943373, 32.213850781606], [63.55535816668211, 62.74458237525701], [117.83096127493177, 124.18822800897604], [121.71958968805451, 130.7717470118023], [118.55578635529292, 121.62009814800807], [142.11432044631047, 135.29386332658936], [68.604674086177, 64.6329099647347], [64.15775645744958, 75.60719422766027], [70.15242949523343, 71.59398202077722], [93.41222157956204, 86.26886050629625], [69.36647544928759, 66.12066505612276], [81.32471416602567, 83.0046742159727], [150.2357889774903, 149.92395069783515], [68.82232632669837, 67.58825809983114], [118.90579861853271, 118.67633342731591], [101.29937109569391, 97.6904238472618], [116.3622364910911, 116.83702568387076], [107.53362867517795, 108.99116821441072], [179.30246519402021, 162.94796011081235], [30.995749691834032, 34.50676667648662], [60.3876734926844, 60.70894804791925], [15.95701666431316, 16.974030230538936], [101.83930394121228, 98.03498264655911], [35.23102818513368, 54.89338179733886], [46.06149177960128, 70.11495448512136], [67.62487676105994, 72.12032905572602], [32.0305979941738, 54.6216177977753], [33.801218016967745, 64.59668646078914], [45.813284387146965, 37.85450071701461], [205.60479042736333, 206.34981382949115], [190.31593327216737, 184.72427770747618], [126.52954963419302, 127.83144771055758], [158.49803794507181, 161.6535681312061], [231.10931682898183, 236.10562016175746], [330.2456755133113, 332.78708086642445], [176.8762925041563, 173.93596532589214], [232.58462995999682, 234.15786435085465], [278.8107711858978, 286.656270885249], [33.16275969600244, 33.7261892976631], [34.09809191100804, 20.575842540817533], [92.18487940748521, 89.91414626594721], [31.18627454227331, 37.432724151350065], [48.45599997508984, 42.23504693125867], [85.53701623629408, 82.90085963751031], [32.71614572377371, 31.9636942003376], [44.21380535412848, 38.40247446338603], [61.26728556864845, 65.86556057606379], [0.0001234540718873302, 0.0001193983729951334], [242.64101176402104, 243.9577730094951], [231.8472127373442, 217.86760347587682], [126.54957547651577, 125.74635590536421], [23.597931803323036, 30.44232066848906], [76.72422763826648, 80.8946330080363], [74.97417222473499, 72.5307261514927], [70.66769717597401, 77.03894216490562], [125.77177190417194, 121.61916101250137], [120.54911708098652, 119.87698074516487], [114.23394067105565, 110.99718570308619], [129.8272505758357, 127.20924564591496], [49.864432635441716, 53.74436068194851], [57.6964294056413, 53.280153907967794], [93.79713934064831, 91.74857275932561], [41.58442940131803, 41.09363912219595], [39.32732437002303, 39.26108630374368], [0.0, 0.0], [40.180916508271025, 36.74167641973004], [80.25623701535471, 97.0457845392747], [68.72490929192276, 73.21876484162752], [86.19664931394487, 81.86092439410544], [121.1382212311558, 121.96044182111174], [182.83236161322867, 191.39682259956396], [159.41974847096625, 177.48680561240698], [123.9651637631726, 125.42053400295538], [168.05021667181396, 170.28029486873743], [219.82730373021417, 216.80866061378413], [281.66709986768126, 302.77569549336914], [175.12756777580117, 172.42109697644514], [218.35552215736308, 209.90837182843953], [193.67885177299627, 210.0886775652534], [186.4997866708684, 189.4701900857544], [126.0610658199961, 130.69002811514667], [92.58843863062174, 94.83679062246772], [296.9736437944036, 296.1998896053141], [310.89602314142655, 327.47650101025556], [155.79487119541685, 159.8240027547208], [287.1426699790437, 275.9702705520675], [279.23936779446467, 260.14252400650815], [111.54624715155096, 98.14191192388986], [98.57787905847218, 98.33352971392719], [129.18000057335013, 128.4563736297181], [33.23406881475972, 31.940382674065745], [94.95382275258027, 80.8163758690439], [23.046685054451146, 20.428126980744487], [94.19337852043503, 114.06344673380984], [136.173043037083, 104.64987348311571], [32.204994013522025, 32.15372550104655], [47.05002927871709, 48.832195984276495], [48.31124374303471, 44.883488832107616], [56.41775774692163, 62.41130564799087], [236.89049556656738, 247.07913318020215], [232.06358677411902, 263.8663835460046], [34.107217521400855, 37.63835129316516], [76.04477927162434, 78.94136636594347], [69.12628036433125, 62.81055721044905], [2.081967767602791, 11.263791963203929], [76.50033190362595, 57.99762488672265], [103.79718865505492, 80.34883397326192], [22.089711694949756, 25.38031754732124], [34.8730390686444, 41.74305632731881], [43.93671977190729, 38.175321013828125], [30.704098412985708, 36.68800551659932], [192.84766521340794, 201.25158125278938], [246.43901824430458, 223.79394747249188], [25.20591253741287, 29.729307399001257], [42.11326886026541, 45.802486314106886], [53.12261327936102, 47.236378109768815], [78.40613074189523, 78.57672010886711], [113.68542443607035, 95.01781940137859], [106.4039923717003, 106.48048411820031], [66.47867618708077, 70.3285633321089], [71.90926805177668, 65.53594961576559], [111.29193631132887, 129.1322060833032], [204.05363243389405, 213.24644745717683], [211.86918871964917, 210.72554939267224], [76.38070453859487, 75.29835858527011], [69.35246272960515, 74.9993133672956], [86.26086207047189, 82.51893351598193], [78.71143420840647, 75.90367694716866], [110.29287182011632, 114.87314029085573], [66.88482865593977, 70.53939705790529], [93.65755650049874, 90.78084331772544], [86.11107732754196, 76.39622628505099], [136.71609451909052, 137.40585110979345], [7.454396079351341, 16.452388768728724], [55.07176251070021, 63.31399172250481], [89.01209298782426, 85.94702225118932], [182.19755468178752, 144.28919742841296], [58.37198059330895, 60.772977759242906], [63.05031432004672, 65.70208796583393], [53.18926052757774, 63.01962817162246], [80.40147848031623, 69.14555484898807], [76.21403273966371, 76.27512936140087], [98.22106225765535, 94.08586530265097], [42.672230432954606, 48.110954203550136], [36.37093674731918, 31.972282562343764], [20.182630738022695, 14.95701883905187], [53.185045271910155, 55.03969667354505], [42.69966506436633, 46.765169189926695], [32.253254085183414, 37.950269167650845], [1.1941151413685986, -3.0806232129553166], [1.2744943249232108, -3.284612684627348], [22.515810306942342, 18.343358382522638], [44.48799451256137, 48.344876439038046], [38.59714179638033, 40.605760329025685], [63.18334093221746, 61.573252196704374], [68.3202230270079, 66.44010742872354], [91.28285463086914, 88.72457740902757], [57.24162338687859, 58.02559303720312], [44.87516010366015, 39.60117964439583], [62.839192877091065, 59.49046962512165], [32.50890584225607, 36.771726221994015], [50.20674231197856, 50.72558453377526], [19.362763371616992, 18.063042881640925], [20.92497967167824, 22.11385384248274], [-1.7943759761267728, 0.47899955213497947], [-1.1985196520121946, 1.7243803039258836], [17.472986555408312, 18.977010806253872], [23.255859510812584, 19.106228496794913], [12.143362477167926, 11.341729544042579], [12.485282703750698, 10.363038224994952], [13.275414154059263, 11.037291442578237], [13.587018298705885, 14.17574015223768], [39.486133487382915, 37.10249959113676], [54.36425127063711, 53.19145762959032], [11.121813624788523, 9.49136418179134], [9.584869841366014, 10.186827460770587], [3.5699527290256583, 5.822898123574532], [6.318952537151915, 5.440987613896141], [6.957000259076054, 7.383506996616617], [12.499877518881291, 12.77205347955119], [24.109486174080146, 26.99453510404595], [17.515709573239207, 17.012061364601507], [22.221546897441655, 21.336215460602546], [31.141268352911993, 32.12387582284279], [35.983762995443165, 30.648835489136047], [28.635238667220733, 29.019692972621783], [27.949313457887143, 33.6311683344469], [12.964454830030917, 13.07402380094922], [62.983380449056426, 68.96839403025626], [70.78841785869531, 63.693796839149854], [58.35418194679525, 56.01085189308441], [65.45784495837049, 67.72380739302238], [53.29038129148449, 56.710345316626714], [71.28466233836548, 69.39123667591704], [2.386847348087265, -1.0480899956619965], [-2.1377144665180623, 3.13095012160462], [51.10112157367928, 56.370562613639635], [92.19029305705934, 92.77715030428311], [46.264912321498315, 44.781139081803644], [46.83668005320064, 46.62057327372639], [47.95050100380026, 50.20870727360773], [48.51068943976054, 39.794653486409175], [58.9827108926967, 56.67406728686143], [65.63212307305764, 67.69194760212406], [14.742002941644216, 15.888186637037222], [25.698539740164705, 20.675979068905207], [36.75876121785289, 33.561423512260404], [27.915563113999028, 25.212804585105914], [13.558225318830097, 14.533900802770788], [-3.774862717899566, -6.977023647997129], [-4.170089229597721, -3.565813899569875], [20.180963364886452, 21.04753614114732], [41.86771224992218, 42.193334642041975], [36.10685016260395, 35.96747817373097], [19.09666986077019, 24.226351567132994], [31.110796731371494, 28.920324617085836], [56.67052594186379, 52.698845207541865], [41.51730034495303, 43.14523204492468], [116.07200176926634, 116.33754214081604], [160.22342955840767, 161.72691162746773], [108.04420671042264, 112.36973602612491], [157.79875757991135, 155.14797354620632], [87.71052866200517, 83.09161409753952], [82.64763950125138, 81.97885983525876], [81.05779984343425, 81.09316922597156], [85.2324993603915, 100.4383665424078], [88.44029589840629, 84.36803177635905], [69.87889109736574, 87.06508820307127], [71.30058104653433, 67.0440162325415], [70.11115557394078, 72.00011833978198], [74.54591897168683, 75.58825736737865], [79.4181485438602, 81.43515028897619], [108.96911853647481, 115.88063829108088], [161.00488139619108, 154.74797555793864], [80.44574893941345, 84.72267764630628], [101.09239592764638, 97.9604293452282], [136.53695611496732, 136.8261949475443], [65.18621171938551, 59.80997664968844], [61.27007692426926, 60.34263886689915], [51.39658143727034, 57.447387699370125], [72.60869408725128, 69.2733548516717], [75.41115957552947, 78.19959891321989], [92.9520382845344, 91.65264533007829], [63.60746433628499, 65.3819618911837], [66.02130115328369, 63.679468022181666], [71.23336537269854, 69.17947909807413], [71.43632434172608, 71.7577747406905], [74.55642317813826, 76.36015747829165], [81.67256737706376, 83.54365228421369], [90.7429791125566, 95.26559165130152], [118.23995637498261, 122.81444109626221], [172.5677474106214, 170.28606187126792], [69.65200098941322, 70.2343651620913], [60.06673044256368, 68.7955026449821], [63.75862707640846, 58.3307393085461], [64.78998256090341, 60.41435001955071], [53.60200422051315, 59.24158124068545], [60.78605331879059, 52.526175812777396], [-2.8092221932782735, -2.1420469813334346], [3.018846950705793, 0.4204687499127786], [60.653372980774016, 62.64834022266149], [51.82983755778517, 58.10345115108278], [51.40845101212294, 51.46355793947875], [50.3813610948567, 48.24889978921823], [55.05178676844508, 50.285031269819484], [57.96729523215059, 55.96739175089664], [64.38739690424718, 58.879792960776086], [65.67969141636362, 67.96010166870826], [14.368542608139762, 14.338256923833399], [19.808637447779176, 18.017651951036953], [17.362671095506357, 24.819170074390456], [41.78781109948642, 44.102616938156416], [41.03599481597168, 40.196195050506276], [20.462859021002814, 21.672888639096143], [26.508044735651538, 26.292112124183717], [35.41989896610073, 32.69948230012491], [-5.185057644501114, -5.125964645706034], [-1.5745354547592, 0.2139953536130932], [18.314308739546277, 19.29531484924619], [24.941078271985987, 27.415929579333067], [14.66900732893589, 26.75227281756827], [50.0891749592529, 44.807709551298615], [177.35679423021674, 174.05420928298776], [266.9872756204237, 268.6145799341417], [166.4632490120751, 158.73273098723], [232.46083729810232, 231.1905965771693], [114.81514045177737, 116.71626611385449], [124.96111714289609, 137.80004400797642], [32.490618773404165, 34.29084726804594], [103.31851875169527, 98.88677266266623], [130.3436722261377, 118.93069198452028], [155.48390285531585, 136.74059638075445], [114.19865935066308, 114.64748336082067], [109.93882273958293, 110.22189624276363], [110.05876872483738, 111.20941328640819], [115.45095956038783, 118.28928837428285], [168.06626580639292, 160.4162262351507], [235.17646953557772, 241.60089590442297], [114.57769941172631, 113.33619074761542], [104.82172028610054, 100.70575228443104], [90.85932220332953, 99.1043528548027], [71.04727159371191, 71.8766865189485], [107.51238789307003, 100.65910072073837], [86.54709649972276, 85.12819218519894], [14.457620199604975, 5.8053302119386245], [13.991698596452977, 11.681560060101276], [45.35116507939499, 34.54647229941423], [95.57259175292643, 94.95138720575], [74.64357226180994, 78.47672006864066], [36.55704226861775, 29.396746523236345], [113.92031790433222, 114.84136048960616], [61.9547176850287, 61.85470004787541], [91.76682051096529, 86.70139936295769], [55.9769033627289, 57.74930002862686], [86.66386350922649, 82.99821454511579], [31.83627954212176, 37.76614826534272], [42.5158535263532, 47.01227331159632], [29.547735909212328, 30.463487183604585], [40.16717630133438, 36.64769545886428], [34.976538497672074, 35.059596785345455], [43.271714440043, 47.90784817351013], [27.022480830860186, 29.184275030281118], [32.53694311483573, 32.70054244188535], [30.953161696970938, 32.86687582696855], [33.74848174360402, 33.14714947489594], [58.80001211257242, 57.88045746182171], [93.03318271703323, 89.43427812694304], [74.13472419873368, 74.098270657249], [76.58392357278422, 73.0288788436259], [10.823678981977196, 14.655891846883833], [24.300277005879217, 24.784473366408303], [28.10377618543454, 28.777273584847364], [21.549444595510266, 20.040645326733888], [44.017438088912286, 47.38304550396113], [47.86354401053488, 51.725710801908455], [21.181357206806467, 15.415245759513382], [47.178482415404446, 48.85151056645985], [59.74347502773857, 63.2251081993198], [80.61165724420313, 82.50792761456788], [25.652987586895293, 25.39562786931502], [28.890281052623635, 24.91007032625147], [11.80256248874457, 11.311759151453476], [82.25172421723676, 70.88750709100744], [81.15221994635428, 74.40021063747562], [17.391382982736047, 27.703103248671937], [31.96577699195472, 28.833382637135678], [30.997006858013332, 35.54112056527665], [34.18221168786051, 31.760042306688], [30.166221654765003, 31.45081453969438], [29.23387098394668, 30.672886723573438], [44.0845761430767, 41.03264608190356], [43.286744204245316, 35.09641911597075], [41.76404351704441, 42.80578929768152], [50.947374297855674, 50.628649352178954], [42.92584472007247, 42.3564700055961], [37.93855260241496, 28.594578977028515], [37.38188731142387, 37.84178525354683], [38.19118452500538, 38.70693730562257], [40.56520226930302, 36.310933518320766], [41.5694131199774, 39.74775765145318], [30.041124702003593, 31.433438096276973], [28.513534818084672, 29.135509867048697], [63.45973426436234, 56.19022723304698], [41.542702862386406, 44.760294487559094], [2.577073789781082, 8.979131829402291], [25.156613486434566, 22.882424504656463], [18.485830037627238, 18.785204306412492], [9.658148104176746, 8.205991407947508], [43.27991951240597, 38.127517930559094], [22.433633145752594, 31.889390184810438], [-0.9452380299789667, 6.94292199477735], [33.40911864132742, 37.030642906105676], [30.20455863668115, 34.91029180146674], [28.793018593555793, 32.312376531364244], [21.44010125650408, 23.71733164272115], [14.694301285699964, 16.151276839400296], [3.176609035529623, 3.7323524017156444], [76.89731273285275, 66.7042826130166], [49.80279502202097, 48.403344884629874], [26.707642490396324, 24.235850756361003], [35.63927177186466, 34.644421384649036], [25.32014664652224, 25.24731909088439], [30.15282394932111, 28.38117248962253], [22.64138789710615, 23.221175631082282], [21.234757466346906, 21.33187275248585], [41.20416139151759, 45.07110157123367], [40.23279607032182, 36.679689463000365], [15.853102268067289, 19.923710836772635], [21.0981521075514, 25.86046644946386], [46.52072697820102, 48.39493889287729], [25.617924914507576, 23.74301172227353], [29.826891304805436, 32.54244489060147], [31.30690104736172, 38.34958930404264], [41.51178856091207, 43.089555473978834], [42.00601231226295, 38.998923620129254], [21.68146958735396, 19.934280966580253], [20.7938766931116, 23.390264111345346], [22.49576811840791, 21.96225932585699], [14.343085262202237, 14.141662009485941], [12.355920844632927, 11.70991028527294], [12.670159277571258, 11.885781892346245], [14.263555827382078, 11.47612548973757], [13.859463577491738, 11.64778928045726], [15.230679276853209, 12.444686207174186], [7.841992929711978, 10.718860089148194], [11.06914440725248, 13.809534566287397], [10.146195112868842, 14.593127356809084], [16.258983986843674, 13.39175285625366], [9.614422337092336, 4.8822024580470185], [12.252260433338876, 13.050415070434113], [13.918421393133372, 14.882751298523658], [18.35089492756893, 18.69740583510156], [43.53292468761248, 48.14155003980409], [32.73443648547635, 28.9103617055923], [13.302934015630655, 15.266273038512395], [28.518220355866447, 27.62517438179132], [213.39445313857834, 218.12603952418192], [189.84935297944838, 177.16179003069215], [202.1592586744341, 197.472721227283], [179.368001501883, 176.31654689609047], [216.35868657050685, 214.53745103584504], [236.70492479728733, 239.751877446997], [275.5594292588079, 277.3447004246137], [142.52109002761668, 144.63423547834262], [142.31382121446336, 147.03422555175433], [144.67008118383558, 143.75268674343351], [139.82327807043413, 132.17731046955936], [19.385620957708504, 20.86293820086437], [161.4044428989949, 161.7555134272478], [148.10616079955113, 155.28080880425028], [310.19263592422493, 315.49877040412855], [307.7353657863947, 300.504310013107], [282.77226887022334, 281.4247678163823], [305.20136221064786, 305.281364944451], [282.74560608010086, 287.84973754915836], [306.41440903342254, 306.0457889595049], [267.7655693086124, 264.6844554769418], [181.3363407926759, 186.83254465828549], [194.15554551685472, 178.15447318155563], [160.4437446408284, 176.0299770925837], [156.87696436247535, 145.03549445325987], [147.32914495629896, 155.11143083180485], [201.3499201339066, 201.83662461793955], [223.8383825765323, 221.10426423428927], [266.9427806485478, 265.69766887099115], [137.80463866153872, 131.235725923085], [130.29419478020793, 125.09104829058782], [128.85504385802534, 125.04990656759401], [118.35324942847372, 120.97209661065531], [120.74029321036147, 118.93164033095013], [160.0744494717252, 159.5114454530334], [153.7646999700944, 154.40856595123574], [181.10783288385454, 170.99694113207937], [308.8330572528032, 303.1690133890156], [300.7436035629995, 299.1673351055156], [265.8504762799566, 266.39909528743107], [298.4013719134173, 301.03982093732145], [276.0134192027938, 279.00886231530467], [293.92980505772357, 293.13061701467103], [257.6152356314406, 259.5983554288103], [176.35875375921225, 173.3240285316524], [163.4894335067077, 157.38709510780285], [153.05561591099683, 147.84841008751], [144.74570854786958, 155.56255135291497], [142.71556053561682, 139.76289779216626], [119.63332702414226, 115.39994398245895], [194.30343175508995, 198.1513222772532], [210.6510166032708, 212.15160780623023], [251.39644383700747, 255.567600321288], [125.15087461549608, 130.4857395172294], [127.41064529102277, 122.66669432134286], [117.63753063676316, 124.7611870591281], [111.96991982342632, 109.59548940269195], [158.82918554723443, 164.60190447925103], [148.12293688560266, 150.88432409794942], [162.09531490468294, 161.23337703494605], [304.3175669219597, 304.0088190788223], [297.48934639226786, 299.3013116705339], [254.88067903804176, 256.34490643765884], [294.011574800488, 294.70121845070594], [164.31773634735228, 173.49724191193184], [164.5552369204589, 164.05591365167635], [160.1857483466256, 165.3651063405501], [225.22618615263826, 226.9518041741024], [89.80584885466772, 91.48298179205727], [103.27351340905503, 108.80413227720442], [194.18757639208224, 182.41008683599748], [92.75553512781642, 93.02093431356039], [199.1523061319491, 181.84850537997994], [86.8634986294989, 88.77436333385975], [94.1207248444259, 98.95001709076647], [175.94698864338886, 160.24541631821398], [89.85440043799662, 87.00007581539273], [208.7313281441355, 192.30055342801012], [89.90386083405258, 89.95044609896377], [101.2874731510862, 99.66603257243551], [186.05694296047153, 184.54102682522102], [104.68306803702538, 107.90511495918116], [178.3836388454675, 185.4891155208204], [87.43478520289642, 88.72680822151318], [91.07375112104185, 88.8845554092976], [157.55736485754596, 159.12681558354393], [88.16232808096453, 85.5788269381116], [142.66932196271273, 143.70819500778725], [95.52013254150472, 93.83595400134209], [80.44036771614535, 78.78233185064795], [221.47460674452532, 220.93982632487973], [151.7304053008142, 155.42693872029207], [165.90287158626847, 165.4927644542251], [210.9214417540281, 205.91394877784364], [171.2500933276869, 170.67943263261844], [235.82700336036066, 233.93573353390025], [162.4944115355204, 166.44835408539268], [182.91728612006813, 183.86658602392708], [228.4110533302035, 222.3450060592182], [188.79185577201676, 193.30275246921462], [277.2929363391949, 281.3786227313399], [187.81437256141066, 186.78579659971206], [202.80563889719673, 201.25475114861166], [269.9540322542116, 270.6552377942487], [189.32492018724656, 190.06872096480438], [138.4552460440076, 136.3712176150965], [105.03059089833818, 100.7712805587405], [91.28660262244546, 86.4929195968484], [135.13818326403256, 132.19450137925713], [98.98692802715428, 100.02161704464962], [140.01084260099384, 142.50665746376805], [92.64421952125116, 99.86367392730202], [85.4931714736511, 84.16120782045238], [132.39657095907768, 128.34359324465294], [98.47452036114422, 102.74614547165982], [132.43323399188023, 129.94700713738052], [95.92277000068242, 94.41177360352954], [139.00860505838415, 140.9339559433081], [79.65457438441501, 70.91377705920499], [64.42641976400617, 71.44942416827479], [124.9300643257026, 131.60080938087975], [73.80173796674939, 75.45564789444887], [137.1480879032382, 131.86432897515334], [78.8562907488868, 72.15482278066553], [64.14512042714469, 65.0460772102895], [120.49942311583804, 118.59477238252668], [71.56698191746958, 75.27724433567599], [160.87279241852752, 157.70896669230632], [137.57041567924335, 137.07788050870246], [121.01637239869893, 121.41345419586503], [160.23944898548996, 159.76019732745772], [145.61187718543175, 146.54872291761754], [159.67335505900598, 149.18127903815406], [143.75262605461864, 139.9899823372649], [122.15455068162905, 121.51413490757884], [154.19585640546464, 148.6584733352978], [146.05553909707453, 142.16964136703095], [191.04202020751873, 191.3668620177429], [137.80393465330633, 133.16378456007106], [140.32290639283983, 137.75901732342828], [145.44397732524578, 141.3678175380582], [302.1809275593874, 302.55539122339013], [246.09527619733979, 249.5790438742312], [251.27446666400087, 254.6197505484005], [309.1012263257174, 309.99057041782055], [268.0552110068686, 274.7268508122988], [304.480815757038, 300.65210511336915], [220.30405325498506, 225.26063725263975], [253.9719490111512, 252.6620104152949], [298.52221730872577, 297.5748253244825], [251.1219835188698, 256.11137743984955], [277.4451912446451, 281.89370412161736], [183.4317455178116, 184.98270317763087], [207.4304883279152, 207.2879012081918], [268.8230266603104, 269.46342774408595], [193.2464364419988, 192.56417392934748], [304.98937886415354, 306.05658625388617], [227.81075639889556, 229.824142213314], [240.54442744206042, 249.74541331225896], [304.34913319176184, 300.6877928039809], [267.57273779807736, 268.6005694058829], [289.1250386167694, 284.1780575912218], [191.88392325515431, 193.3424459171101], [216.80453766895255, 217.31662101620452], [275.45368428883, 276.7758230890359], [229.00296970209666, 225.95233891310323], [295.52731075632653, 300.0009151586355], [213.94832597980516, 219.95120941249718], [241.26017381785093, 246.35687706264758], [296.4036696628369, 294.3035667660938], [256.56552897207774, 251.68149980451898], [268.53210568222516, 265.42352137286275], [176.6391375056027, 174.5150648150081], [215.14550614269749, 211.36121134977836], [258.0309650129486, 260.45748196786656], [206.7753178258031, 207.99041207782727], [169.89725745458946, 170.56905471421544], [193.36363101808, 206.29074399467402], [86.75811624130597, 88.06437237223177], [84.29340807632758, 83.65405649971828], [186.54549185722107, 171.73314750173137], [84.6844755656866, 80.44759611864345], [209.28897211003246, 211.3299900276421], [86.44864522866672, 81.08189239911096], [87.74866009011899, 88.86257537127668], [155.10298799480137, 158.8372450209728], [83.22552050582237, 83.95421084409105], [205.49058394513548, 209.46510817692683], [85.30965843263945, 86.25508136450651], [81.55170524118464, 85.8913111906995], [163.9713770781495, 163.84651884670444], [82.84356976052453, 80.56205698588383], [188.5170706133847, 185.25611781961408], [86.66436733816931, 88.40695517064589], [81.75902832768813, 83.3590483700458], [141.82007209365386, 147.38252122464257], [89.12471299876097, 84.02607214825454], [147.01392825846912, 150.10212742317913], [95.42079071814976, 92.38617702765825], [79.4051488256764, 79.77417291843088], [219.13675678770386, 222.23361844045007], [139.04046828172443, 141.2965353583321], [151.8961854405841, 149.04869468109547], [198.364333467684, 197.24186722841944], [131.55294843722183, 129.52729647756996], [241.2401896023158, 241.73685776571682], [153.33728863746333, 154.65089679206253], [167.57632012170444, 169.1511285592146], [224.17866693223988, 228.13064072051066], [169.99852111568396, 167.27791518038083], [278.9322579453257, 280.1303050480403], [173.30864598574271, 173.97998690473912], [189.48277925276574, 190.30421507526358], [261.8805273944191, 263.83770138618155], [197.39897278026032, 197.00132447545894], [146.00767217470047, 145.77799854073402], [93.25645177365517, 98.12800742263818], [81.30003348223428, 84.84689549433352], [131.22387890367276, 129.89612932138124], [95.64555686193174, 103.33759610536603], [141.90053812493005, 134.84464835075164], [101.23620267548705, 96.96967406981781], [79.78921011498339, 83.10584581363747], [122.37896518601121, 122.89931920299173], [96.84663603444064, 99.7070428251871], [79.12550778106095, 78.25218477454763], [124.0494980595933, 121.07038444718353], [89.4818584533474, 94.69469947805523], [139.9205007321023, 139.06803223465343], [78.48605367875166, 71.96111567493739], [69.56698293495424, 66.71785001935501], [122.07656927331391, 120.36998821016682], [75.42188315825516, 76.19883459368017], [138.3427412288472, 141.26497152809657], [77.35206373976746, 69.39841328155022], [62.76661126284934, 65.88520014808421], [112.53013748612497, 114.41833062253602], [73.44017480695155, 71.29527208585853], [161.3637157205372, 157.763907919989], [132.25765744282583, 138.58579018812895], [114.30165603714907, 116.59370602919924], [162.35944826147852, 156.96387925938296], [146.33310934215768, 141.88674821271647], [148.37597644984814, 155.63327778152535], [140.65716491796945, 136.0486263666706], [115.14980754002806, 116.68565507341603], [147.80747549443691, 160.40568077179216], [145.50631934759195, 149.93166118337007], [186.01922002676008, 191.73680017733395], [140.96630475822343, 139.81812837869398], [129.0191752675061, 130.0995651676804], [164.17333001480745, 169.31362915266985], [139.76135441798067, 138.93497586594236], [310.23762722054937, 309.3526917866093], [228.4413431521562, 228.33248207066566], [240.3027073751386, 242.62036672149583], [308.1798067635764, 313.32987462385285], [245.94186834530942, 252.3593275527195], [305.29221823868863, 307.2679321947374], [212.5640074155984, 213.86152637796044], [228.42811510256487, 231.7503314238521], [299.1544021079119, 296.14664041152776], [231.83853334325556, 236.75490832250682], [286.01772135831266, 288.42163542107744], [174.06259538084163, 171.40766791763065], [189.61208838521023, 191.07862054246067], [262.42156084682927, 265.0992545314375], [157.81396546909556, 155.74635056231864], [305.6091469182642, 307.8286239762375], [213.4210802086632, 211.85067373434129], [225.90880753340215, 227.33858928610658], [296.07501792772064, 295.74650690182045], [232.58923828236976, 237.5990249000978], [288.04674885775455, 285.9740381940305], [182.43389031217555, 182.40909046324788], [201.2370226928199, 204.28372935947394], [274.9685903891786, 273.0250460684014], [206.90985438480658, 208.99180814297495], [295.60433492780396, 304.29465758837244], [204.93731890595947, 203.0180794061036], [221.68298855930672, 227.08864226608233], [296.1738574054224, 297.56755354768023], [231.6401871010816, 230.4472335878977], [273.515527872839, 278.390612648132], [168.19641916383412, 165.3045808576092], [189.17972058130488, 187.47929931526483], [252.1350889313041, 254.72232061047313], [189.04047942152522, 185.621171251691], [135.40859285231986, 137.04174462246834], [197.94395565415698, 197.26185913332657], [161.26646046250298, 179.61260552386523], [84.43579436369532, 84.85090535088364], [90.82862017645346, 91.0876042217775], [87.10492391546035, 87.69660353696769], [149.2158940684196, 165.10256156427704], [83.86569253762514, 80.94801898022092], [88.50831615496392, 88.29009126046051], [86.62141530004686, 86.29071557978756], [184.89176955521918, 187.99471882885095], [171.7272162481784, 171.42544936311498], [83.75690158332971, 81.14663382092786], [86.12633055230054, 90.72865662634399], [84.86840080964194, 86.17577500957583], [152.4796314329373, 161.29879308637499], [147.2439613949556, 148.66886176550065], [84.04774674030577, 84.51370280719914], [79.92256312269257, 80.79162500955911], [85.95718291114932, 82.70643103391379], [124.26432745384199, 127.8125325028139], [205.71677395684634, 204.75274250836162], [204.54090964058796, 201.45951740218675], [142.41887858768558, 145.1384796692684], [144.05349549183768, 148.00458239725702], [138.92608064703757, 137.98495535968698], [227.57869850536918, 229.055020358565], [226.09497595356703, 229.58026310000258], [145.51574533906646, 143.20718779533323], [158.35059935147532, 161.87832232147215], [147.20478148342067, 144.63329331312326], [270.9603087643034, 269.8610450700942], [258.6729637108447, 257.5389646881991], [170.58819846461262, 174.0585673190975], [183.00031186857723, 185.6741510409483], [166.28609422322955, 171.02106276988667], [135.19689255781387, 141.07865556690257], [136.30779492123787, 133.15526343507656], [85.17744291124097, 95.46499963759567], [83.42402156441818, 83.24334839073211], [90.62501848043104, 95.09255928155201], [137.28294665357004, 131.4890464365143], [129.21332681976497, 122.4969733276472], [98.97126340594895, 95.34158641876596], [78.13486869895947, 80.76990832311816], [93.57119736056927, 99.66723781721483], [132.52420046820072, 138.1053518796757], [96.79242690794865, 91.2286579369731], [76.4317098340236, 78.041437829172], [87.15634938013386, 92.29940118357888], [131.74733006987856, 130.78749902127566], [121.33917653548167, 123.08424038409191], [76.74824083583405, 67.68131452860297], [65.35414539181915, 69.1655724774904], [74.63942870929573, 84.45198886812176], [124.98391850847881, 119.50326229926338], [111.97210866096816, 115.0013759667025], [69.50651967489341, 76.11816942729155], [66.12152854400887, 60.09397694660382], [73.27568898704274, 70.83702861299507], [160.31427558189228, 159.81160186300494], [159.42967348167616, 156.9565005174224], [131.76994866327732, 135.31550950920737], [111.10310721504958, 111.0380229096564], [145.26078384617153, 140.81365063739364], [153.85040824976767, 151.93351388330282], [147.82061260595833, 154.2723905673739], [138.6011074656252, 134.61932772842766], [112.3377649399458, 114.60536765621555], [140.66800959344263, 141.18123909582533], [172.806957978858, 181.184489888757], [175.7667846017038, 167.34086015580527], [139.59562339552895, 133.35712337667093], [127.59663670954734, 126.56604961411782], [139.01097998331392, 139.54100147765473], [309.40450466924074, 310.5706346511492], [306.95154136432836, 307.8937702146719], [221.7262694811706, 223.90009815103056], [230.54461469542113, 238.43419169892297], [214.71600398854446, 220.22809885282823], [301.8247912950527, 302.62922303151964], [301.978121487095, 303.9088311995144], [204.1088290389662, 202.3781130493326], [222.69202040185405, 217.6387999613194], [237.5080758733012, 236.69748710689424], [268.30629137707876, 271.8593030082877], [261.5291863864969, 262.6106296059581], [167.53808878526237, 165.62983138242936], [185.6685275974968, 185.6292246441044], [166.50508368791515, 167.65447821279105], [301.4635771410899, 307.0567652920962], [295.4282506417761, 303.46061619834427], [206.93407092607615, 205.0511865253582], [222.78770087324298, 221.8041678957987], [238.50310292092453, 234.25006786047993], [278.0062831046337, 278.3868554874936], [271.41241085016884, 273.4988561672117], [175.69780884777407, 180.6955110405038], [195.67989912304375, 196.9250613924454], [203.89485658090794, 202.5541973779646], [295.74347441606574, 297.4572406270879], [300.4782328191435, 301.9244080899082], [201.77510188780218, 204.34647798088417], [222.42947160129273, 221.54270840522628], [226.08750471663342, 235.3671991706987], [499.271109710085, 565.2011165697593], [450.6662481455764, 526.6295981994266], [392.1956272964095, 402.32829749734157], [341.8140798032591, 388.69508034692535], [491.3241028371976, 482.8619868998715], [163.46061017898427, 170.72524355069348], [144.04947496174287, 139.84618278293397], [158.25198267582977, 161.9770719030452], [139.2027415110451, 137.17135144288505], [126.25204908249142, 126.55207072005354], [124.33484937101738, 123.80042176183889], [139.66247529395076, 140.177671683018], [137.96531943429017, 136.61389421790648], [138.4514981240593, 138.2484952915646], [134.0427812509397, 138.13766712431965], [124.15026684862949, 124.33094488792642], [124.60744133069782, 123.06247650235906], [132.41094875642568, 131.82100016548355], [119.89900011883182, 125.95488203179661], [121.7694191219937, 123.6778882351389], [121.99098823088875, 121.67067858299258], [123.67644811720682, 121.67033021561366], [144.98149686550857, 132.93459250351793], [116.14311796097883, 117.0319707000214], [109.46903877090752, 111.79886108089842], [116.4958929006682, 110.63634688963343], [124.5941861779736, 123.4680204345797], [125.67845940660116, 127.251935985177], [132.58433353466182, 132.2554389339381], [112.87045956097964, 112.91218731402586], [114.14584339422693, 117.89130778989582], [108.25926702996037, 110.12185430973886], [118.42139258424812, 121.16707933791457], [121.94830323696323, 119.56924710895576], [119.24390455886203, 116.19434335370215], [141.91200748531764, 142.1918977558579], [141.33134635342856, 143.92288139343532], [151.64714528415394, 148.37882349362312], [132.23917138536888, 131.31113373478996], [125.7564623327986, 128.51408514302497], [132.49570311686392, 124.51705813682128], [132.7251144754612, 134.22826617542347], [133.976287887872, 135.65960558399306], [147.94291066736062, 147.4474484071002], [128.0365225112779, 128.66018056784847], [123.29009215884031, 121.19339526362012], [117.34000477687272, 117.04223090132365], [128.06101442591313, 128.60924827653065], [121.2929605523956, 125.93391193667344], [119.8132792171894, 124.19258047843155], [149.09521422667177, 142.95072881625845], [136.6600375745015, 136.9084316541797], [152.11641244219993, 158.21299767704465], [132.55979766687616, 130.6606688794212], [119.74674096270401, 118.67372757523535], [129.14047567308714, 126.81611525257192], [136.43783881605484, 135.07269810361979], [133.98503267991117, 137.43030541612458], [138.05923927685123, 147.10882278302745], [124.42279917422866, 124.50938090899452], [116.54043556161028, 123.80930933810347], [112.84604845361491, 113.51927609703148], [128.29660588472646, 128.0879954503281], [127.04302002907644, 121.99807516756239], [121.12519093374164, 124.0213881496602], [108.5548428711698, 108.29167998395602], [129.63248543134392, 131.81569845605418], [146.50139512349907, 150.20970977738378], [58.56952450219937, 56.811651488332224], [71.18241149168364, 68.21966689828517], [69.51045318673357, 71.60493314873491], [77.67422466252626, 73.65729640789218], [71.89505882862373, 70.35392476486739], [92.71276677828965, 99.1193093655064], [38.389491582341456, 35.609398723185116], [39.756443573905486, 38.11721999263156], [48.805041934737346, 47.49549358535644], [22.928925172024606, 39.02083518105146], [39.567558641705304, 39.69759340070088], [46.915656330964595, 42.00255237587224], [25.00129473113273, 34.04651537634474], [39.794055452821475, 36.604694816568], [41.83420367800626, 42.7501360163634], [120.5635404519958, 107.3025603862178], [124.4084503240597, 104.43444027364328], [129.2428124876219, 118.60507491644931], [58.369674430694715, 63.13604841458714], [55.948013956506, 54.19472880291228], [83.79305800362073, 82.24702932099115], [100.1672345587254, 89.81665777970858], [107.94856736117094, 93.89867308819238], [95.66113780849386, 94.85929030628864], [96.66474943473912, 92.52684084754725], [114.21241819775503, 109.3801176027751], [107.40679975092826, 102.56168392835306], [103.91083304156646, 106.4505222270478], [125.25034108890361, 124.5197115943726], [138.9141027542646, 136.03177270170613], [57.36158734054996, 55.8256499834904], [69.1458358695698, 72.67547774657515], [81.6194493338468, 71.67468976083924], [72.36805347678607, 72.06283548495058], [64.1580577280987, 66.23759526335164], [92.18648089297851, 94.73685772715321], [37.89077679727263, 38.26217733013566], [34.373670856689486, 38.69070189799011], [42.6137662152278, 45.31744262581429], [35.458131986453026, 25.581333751190435], [39.28792299352721, 39.08791762311797], [43.04772090175796, 45.89770189197979], [105.77420299977996, 107.8560923770297], [117.33108624756505, 118.8465152995717], [136.2936157590358, 138.1253715055003], [57.76898658680987, 58.3548401398717], [71.2359454307928, 73.10459826040702], [69.34301572446712, 79.97204806423345], [70.31087254900285, 66.7291891306738], [62.040485069362234, 64.09922473958699], [95.51533281819917, 94.5624534248618], [34.790419805051286, 26.913123769441228], [38.89869574198411, 37.731404012684806], [46.00636761497921, 47.07888271513222], [23.81795585683723, 33.70021205545052], [39.21539413885118, 36.45008265042056], [43.03232789831999, 41.57217461806519], [250.86795826335228, 255.11313453250915], [181.7192640481283, 175.7034569813639], [174.23870519669356, 169.30560752864972], [167.73130806957926, 176.322254917448], [174.89516072198944, 181.97920145796772], [164.69657069056825, 165.44380676779383], [176.9329401628145, 177.04004346113342], [171.16601551558153, 177.09715072803132], [180.61329048445526, 180.57990782736562], [239.53140179987008, 233.52926422752057], [168.87691716606375, 159.68824717471637], [163.17415398099655, 161.0901080218956], [240.1774978294312, 244.20842899471563], [156.70112382531642, 173.03879213369828], [163.36930867616692, 170.42245833512834], [158.90990958332955, 156.64241491149266], [125.65809399418922, 123.59123628577782], [120.34027098123252, 115.9138335755709], [194.92980171236957, 188.7610148228954], [176.6659972057714, 189.0619339045141], [175.24745540647126, 176.53500174022102], [171.46538426657924, 172.20361578929015], [179.74111051285283, 182.18880496591325], [174.32064430293585, 173.64901870389843], [162.85868556974913, 152.47586378711256], [112.60352713059011, 120.62131417930118], [112.60914894622483, 109.96751614196172], [167.0078224862578, 158.24124712680887], [119.79650664009498, 123.36893378448866], [113.24253742602686, 116.31048875119136], [246.12592742412104, 246.35201610666485], [172.0699230931433, 175.00022637574142], [175.53194397937014, 174.36880927958788], [183.26943136167694, 192.27845370949535], [192.8107850683482, 185.0841203614724], [158.9558429880924, 175.6240729357463], [176.6447628223242, 173.57480438148306], [179.04373307668874, 180.64656708479694], [179.07297556356605, 180.36960774437452], [171.29439951255517, 168.22417425671298], [123.01684885103607, 130.3651245861903], [115.53426091393705, 116.3747747450785], [166.45684603324867, 163.08798233356194], [133.92282757172683, 133.89421089757508], [121.81901134652436, 126.25546227229106], [241.2059202019207, 241.54489659597846], [168.67867357656885, 170.5680779841918], [174.9433297904263, 173.43435625250652], [188.61229899682672, 186.83414754438022], [188.5500061840774, 188.90780626861297], [156.29835302987271, 176.27176792418442], [169.13151740719445, 176.48588794984204], [181.63620870693734, 181.07709951080932], [180.5494041188107, 177.517482099098], [156.03513991358113, 166.1002462092159], [121.80060889166657, 126.45076881431763], [123.69171345436467, 118.29332076328214], [157.33857810743865, 167.51467324027988], [131.52253479526206, 123.61742171650228], [114.83822774410038, 117.76930959822568], [104.49687151170775, 106.37779453116582], [104.57702768369518, 99.0022283911381], [78.4340589143888, 76.51529621352793], [49.43992434407599, 59.0251970815076], [55.70476340776, 53.430851260008495], [131.12767795535962, 125.96640878567383], [226.52569160583712, 228.79604853265607], [124.54392191298848, 122.36096467736513], [132.9376503353423, 135.48494544362893], [147.89893307620247, 154.756790214992], [158.8616921983271, 135.93656859353192], [124.86717323275042, 125.41486933403019], [126.50807041239224, 124.32307684071704], [120.69591399610829, 121.75281714061433], [295.3311124965093, 295.2239887929996], [151.24492946254094, 153.18596618697535], [161.79318269652748, 159.98278709246028], [213.8506629645666, 216.53642514391981], [196.1396390649339, 200.5465034927843], [115.19907110802063, 116.93677059881679], [117.30001507240411, 113.8800040237802], [133.16053132672369, 134.2933009460292], [130.38963025075762, 130.44783689020753], [109.18731351894814, 107.88313579436024], [107.27660948557971, 106.16447444365889], [108.22300524298453, 106.5977638252441], [308.092498705022, 317.5674997270347], [146.02276933274777, 155.2318803778823], [148.61353274690578, 155.31289518264504], [199.3276855326315, 205.55736447528278], [221.90527939608342, 221.3892097907607], [120.4395898032979, 117.16030967011108], [124.69875821916784, 123.11720012984392], [146.02268591033192, 146.02565645596206], [141.96914696646024, 147.67325899344348], [115.38871512061678, 114.45950608346118], [110.52847055727044, 111.88242024835222], [114.90318311031905, 110.97777082201299], [310.2562208037119, 315.34540861531275], [143.41764624657014, 150.8138746030731], [161.56293699954284, 164.54776753527057], [203.78229769183588, 204.19743304031695], [213.53164264697116, 195.66584477631594], [118.59561558912173, 114.87801091562324], [124.20257930498713, 120.71521415458443], [142.93904172487595, 140.6734141740174], [131.18828372138705, 132.15032604828326], [118.56102030545665, 116.72836562989728], [114.27846252050853, 112.43061648141736], [110.14742505586837, 115.28266097632984], [312.2697572787094, 305.6193313365642], [147.31798368410335, 150.8078180586514], [151.3976444927896, 153.02991172623564], [205.16510760244896, 205.09367788536503], [104.54497467554175, 101.58909174239417], [195.94267553866564, 194.4155651802265], [110.9502426334443, 108.74515309667586], [113.5014667948783, 114.20761795213605], [125.99781707824413, 126.65597844161519], [130.36775592173603, 132.08114070752598], [104.30051478664814, 104.44703623389202], [102.02372480359652, 105.23530260817971], [101.78892257971414, 101.0331562512633], [310.84464760101264, 304.64400876023967], [142.6663423026003, 143.94002088986036], [156.96594510806608, 160.27999448770134], [195.78280652807214, 195.12443380096911], [109.9704373237837, 106.21058098721431], [209.54300071340964, 205.03249661414094], [113.44366634778146, 113.25505622696176], [118.14288468301362, 118.52695024568939], [130.20813262227793, 132.51035822069394], [126.50663768538979, 135.94742978673838], [107.43338985709386, 105.23786234765703], [108.06826629641535, 105.57129806006644], [110.31581071952701, 105.1628129327544], [312.2823555940699, 304.9034062147233], [148.94365103770122, 144.17571282444374], [153.46289638660608, 150.30562598707456], [199.9016455816188, 198.89380610102646], [76.72422763826648, 75.49668819563158], [191.95610695575817, 187.60445676904925], [110.8886414919503, 108.91282019777759], [111.18604225103584, 115.12663395191595], [127.19851797730797, 125.98370842665423], [119.89800932072592, 128.89513745445376], [101.03560549616887, 96.69070126312081], [98.2439446616611, 100.89232807464998], [98.77163573104657, 105.0669028721018], [311.4735525294017, 316.76871292329827], [145.1660827143881, 149.589700462926], [146.45091615752312, 145.66573778006583], [190.47069801416137, 191.25967795622375], [67.63853605398305, 66.7718689409096], [70.60227450710829, 67.43165058699306], [75.3832541881549, 73.08533371782973], [54.706798064891615, 62.0171968614182], [63.22051544141246, 63.60123794405651], [70.11708466244652, 66.9022677515788], [35.49573555313037, 36.6981284743696], [42.36768113663318, 39.37751266277907], [99.48067435466791, 94.41852429858574], [36.8589280860202, 38.42398483527057], [37.093816202191036, 35.33273613678183], [80.76852554630962, 80.44560318274489], [37.87131108784373, 37.112755419711576], [49.405201936061644, 44.94228550761048], [91.73695927786659, 92.03120951862122], [44.51208671903624, 48.920358302591154], [39.52181894273539, 41.18234309349518], [78.1853434020835, 77.07718020115408], [83.88832641870296, 88.63265325713186], [111.78316268718854, 117.1372389593709], [46.953163686023316, 43.93092544631739], [28.686989816036686, 31.229959678813977], [32.88994168124847, 29.254245775964165], [45.79563108965519, 46.79026827434994], [34.73921302891718, 33.12252213714093], [41.017480516563374, 40.55526858255416], [29.470635644379776, 31.400627836650166], [56.735361790775016, 51.703096869355704], [53.6145661043781, 51.178594671832705], [41.42577844053724, 43.111868277959914], [63.61688901906399, 60.955211864733556], [58.29184329687163, 57.11572352195637], [51.96258729080619, 54.57054116831407], [92.38194399188481, 88.72808936918167], [101.9785408730072, 95.56200429972635], [123.25162548680754, 116.88917062405974], [86.32267497061093, 83.49672797232708], [75.10243361398278, 86.49555773785724], [90.91881616740577, 92.4289447558651], [50.339314552661534, 84.29341925609253], [86.0801034506847, 116.89451345279433], [154.2590616798472, 151.56906178409054], [96.25146831348212, 91.9408431300879], [97.80978564771054, 96.48279362043675], [93.59403384228017, 98.97197814260818], [50.38462549321895, 78.07640213141426], [101.71878067477579, 105.60911428642851], [105.73925496728359, 107.93173325823426], [84.75370262487931, 79.95992967861767], [104.00332854961637, 105.61391558258119], [107.53215990066474, 108.1066873847277], [24.449609333506665, 27.40569139183006], [40.29302854050236, 29.754877719339994], [17.303491607953106, 17.893411017516158], [85.80368181837967, 87.14373835839785], [54.568129546087036, 55.11327930403664], [91.36179712445605, 103.02930265699277], [72.33365787873844, 68.78865524412097], [79.31736742062836, 85.19126792436586], [108.11305654413596, 109.03798136752859], [85.92527312944253, 88.00969194002043], [96.64299143098104, 85.62206466632898], [98.80246308929469, 97.57239802955985], [105.79322834002132, 99.86635495872798], [139.60869985229024, 131.62553941853446], [180.4290475051749, 156.29490963308734], [55.357532088409215, 53.287458237564806], [44.15481032214623, 41.663057644669415], [22.571332428174195, 29.392500669580485], [82.33327529386179, 85.39427471736064], [116.18768843205123, 114.62072161800063], [160.78145747596346, 162.2485712590768], [67.44176749737838, 71.59371935508375], [57.233870408084044, 57.98873371411], [114.39860736384665, 119.32971440117642], [48.67715547148862, 83.45918437920186], [81.3799004552719, 117.41665289357813], [131.12943381182785, 150.47303096897508], [112.25331311717036, 119.7295854674289], [99.94577464188643, 103.05300288517375], [155.5259361306462, 152.5485968860654], [55.49836203270056, 51.82923515634238], [57.0110396418375, 61.14211047122538], [128.1557363499902, 120.14444422713711], [83.78006649295278, 82.68583577505896], [107.26418865467951, 115.88149061965453], [153.44555881112436, 153.85955994545577], [90.85260147277234, 86.11022361764276], [97.098786748711, 105.60311395966784], [129.3202816679903, 128.16977498879004], [8.288616681374975, 7.17721084826244], [6.486110639806142, 8.888933031892876], [8.624720489113464, 10.15099229560552], [61.47721193107425, 61.64040141384935], [55.285760861840345, 54.446781742152794], [46.044487461980324, 52.909867276044814], [55.080740623065054, 79.01306767124865], [108.86021313498051, 110.4167588278883], [112.58568799374832, 106.48475202490033], [30.203269507672836, 26.351069432313423], [43.721963886533935, 43.59539582714933], [22.973045177225337, 22.66337408748059], [6.279383371157593, 7.081836658080995], [6.512590162367786, 6.755720142985261], [7.847762798492017, 5.902900946264555], [58.03115498503787, 61.863599929027316], [57.78035378380683, 53.83420245397298], [45.20736850139478, 46.537100596257666], [49.505151057074414, 78.42957922015167], [81.84142254395368, 109.89853035327842], [147.52922808788443, 152.49278882718158], [50.06322286041971, 50.489997236825566], [38.89479506792671, 38.6475298009063], [23.443612731566287, 22.411779040487914], [63.577773688776325, 58.663277743584764], [78.90453477814427, 76.02476536470914], [100.09254109867544, 96.87639712290252], [75.1211330786841, 71.04711982882174], [84.83312142101175, 85.77318612113987], [45.22943803156518, 44.09691681083564], [98.55028132033493, 86.18700435403592], [-1.6361613993439565, -5.29110662295696], [-0.596603367616872, 0.8139296669768044], [40.70268937683106, 53.49377720111532], [120.52671740347526, 99.00382065432431], [49.698425179338685, 45.25919834542272], [45.1970315384327, 45.70384698130613], [43.391155951789884, 41.89636240885041], [43.897236237201824, 56.98541722163381], [76.46739819569433, 76.78036865646422], [87.8639544454727, 89.69920797632176], [57.79529879492924, 58.72859066624065], [48.773623782506576, 51.95420432518273], [37.640886388534, 43.03074332507566], [66.31304852433516, 62.75651020623832], [55.62416229938697, 58.48331097152925], [37.46991949768321, 39.274859594184505], [11.307027343746995, 7.023568322830599], [28.019336761735474, 32.170108462848646], [60.26613201393738, 50.409207174332835], [56.985335669987876, 54.44822313203568], [47.76850437279879, 47.05824389318829], [37.20297130793904, 32.88896997320978], [59.887497573732134, 69.70603494728294], [72.02305453188927, 65.1572501123232], [53.52228699374433, 51.6279930032469], [47.77452003041912, 39.489546872035106], [74.15001809083671, 75.57680073893096], [27.247460101753575, 37.004791348921714], [54.97971895262029, 51.12678316543091], [2.1297083043479006, 0.1097146573573724], [8.320505243147394, 6.2828168207125685], [3.0511737895488693, 8.285573537319522], [2.0665946618202895, 0.9721610117646451], [-0.8273513810073041, 1.1458223161908307], [9.334933476984489, 7.82421941484148], [15.601121482448775, 16.816133863845707], [11.529201395072334, 10.353140305643201], [11.651736939901383, 10.467841948190866], [4.974425504445662, 3.730854978649802], [38.32644586634359, 32.92061759422626], [64.0512936265568, 63.21784615676066], [52.37550142949962, 51.17408887695308], [49.11764455132976, 41.91932095730448], [37.972252882375386, 34.6591468777433], [33.183509630618005, 29.64078886779751], [24.7458249597101, 23.02841811996791], [31.69700610247552, 31.489530119982255], [31.22660852425841, 32.23677800896023], [28.735931330884306, 29.93737798969113], [22.69611377206303, 22.83147791234566], [26.086902012656445, 24.209397150673695], [75.71851627565033, 69.54564087522772], [64.44426943493687, 71.65189247067349], [18.712292426842225, 21.634540390135612], [61.43351724385167, 57.51399144390338], [70.06081526636245, 65.01896385922188], [56.28650391769601, 50.91249671181633], [69.08186212924727, 62.893242081484175], [27.601753046019823, 29.678324166229373], [40.79769868075239, 44.60519803842409], [-3.615349918094058, -0.20007519249950434], [-1.3591207294291845, 1.3823539827481426], [28.71999239137918, 31.868386551430326], [43.63037486961616, 46.380291070659354], [28.306006660067446, 27.60177669495161], [30.60815789095504, 29.869014787440847], [26.321079729934425, 23.032684742128524], [36.15658738406893, 29.15142631306935], [55.118529088431586, 60.28982442119127], [64.57071918579602, 63.284685319879344], [32.331954635828595, 31.00863103764965], [37.59090364393791, 37.480458227732925], [26.484989545372464, 25.095607573033192], [23.126375616009888, 22.903327373863505], [21.808111861391865, 23.602825227752703], [35.31371883931996, 38.4032557179724], [47.32987901324884, 49.43878933576207], [31.653593402073795, 27.519292152732035], [27.103710250680365, 28.750384358869525], [49.27030719917508, 51.35912536850259], [53.404760295927986, 54.321643270881665], [29.164313498646372, 27.642428063866305], [115.7326644222672, 115.56505845440171], [161.09563428043262, 162.10197935870093], [98.48344879106189, 96.31582708277224], [147.3354716299207, 146.00685855083805], [49.19825655648525, 46.490779131447155], [75.872902115492, 94.2552734376005], [45.74728468763715, 42.29642195917178], [77.86871837215924, 73.1987547836542], [51.83144933060187, 58.06121147533923], [80.17951984294527, 74.2199605280718], [43.056837508141086, 46.02078974918349], [51.025175026923854, 49.037963597479184], [44.92226987973834, 39.485538942860686], [60.33116065275959, 55.96985680105452], [103.97662966641701, 104.45288577646353], [151.64891689179325, 150.78086172383297], [54.32491068938253, 55.48512272039952], [71.54687066453832, 73.08606115068451], [97.58983159881653, 90.54841167174573], [34.59835614321211, 38.98563155030463], [43.36852500128172, 34.825499822396104], [59.14808525213165, 70.93095844443604], [45.53975070671538, 41.58659936441008], [51.83451378137306, 53.30296865813], [74.91398031332426, 69.86555378676377], [38.22901690398082, 43.011693238027995], [45.06655625763639, 48.26533079813968], [63.753492841937295, 61.97975632181697], [46.51871763634523, 44.904431345865746], [52.382262348285764, 45.78102672918226], [55.47734530601406, 61.80589721858294], [66.50563444859334, 66.5541253109881], [97.29365596734604, 94.70818007720509], [128.33543097507692, 126.17742174269618], [33.761762236574256, 35.164889426908566], [52.860701049345835, 58.432394543782635], [46.563074078635466, 44.69822607414761], [62.01857385569187, 66.26633778345459], [30.704800288292788, 31.033270719131153], [38.14020949908257, 42.3659314876065], [-6.640560619428214, -6.400398290971836], [-2.419690441513275, -0.14736460788942907], [30.799306555492265, 32.08726990999389], [37.803116106504504, 42.530609177269596], [29.330414939207913, 27.97808434763767], [33.953734701517874, 35.3081357008448], [27.903956080380134, 26.238646843501307], [39.35629836155594, 39.6799493996094], [51.632689259261326, 51.947778780904486], [64.43140819961623, 64.81244851371281], [44.92291793028028, 34.65592171491909], [38.96314006086757, 37.95562168483923], [25.847210409031874, 27.49936165555276], [22.09035715689447, 24.93963659837479], [25.33904025830546, 22.57549250973028], [29.351479011511024, 28.25483849606877], [31.033700802589102, 31.128728770772824], [28.128145846622093, 30.663169688954554], [18.66582234894652, 22.181424541941404], [28.791194145190374, 26.44350603500476], [32.63442561766415, 31.844651053068162], [55.23284594387985, 70.56046219904437], [77.66346217542025, 74.80326331775001], [29.08990074071494, 28.56222290386316], [161.73267017826973, 163.04581812509736], [243.458247158341, 240.44053461315372], [142.429021105277, 143.55162079995785], [197.8035423047666, 197.6133258522346], [100.18846819600995, 102.13481752400767], [147.88549047160208, 135.7558030906347], [17.08137102154832, 14.2569577544146], [49.94998778067364, 26.915411664609053], [104.84290706880084, 105.38392366152621], [159.32328710282184, 145.71654640572885], [88.0900282748809, 90.13776731339507], [89.44822702737983, 97.72376162658715], [83.96662345700702, 84.27367125504577], [102.79032803951092, 102.56423212016318], [145.065288514779, 144.92454401064123], [209.4491048704861, 212.44404587269014], [83.83283866901614, 80.50928337064283], [73.17743772192642, 75.34442965586662], [56.62225990572301, 63.113673600300544], [70.24269717182457, 79.49044893166536], [77.02942101799181, 74.85195599317086], [64.78112675082383, 62.8552940174049], [84.20184205569012, 81.20275246615427], [76.49122156245383, 72.83982552586673], [57.472017870344075, 56.99527494231552], [23.901592942729053, 21.01668099861854], [35.03738006520848, 34.58636359607448], [72.6746277991827, 62.64798694143602], [82.84501558042157, 79.24059074443664], [67.50342810814237, 68.2836505516284], [47.841632166447766, 52.305527389277074], [88.49648856760268, 93.22364187388777], [58.814407009460965, 59.642157024049425], [99.9402958489951, 99.32391872212398], [46.92403941117838, 49.12778590903471], [88.94535145666714, 78.73392232561142], [15.594982524263425, 16.804244078042853], [27.71701216313668, 27.234585742315723], [16.778308357681397, 17.33344986659756], [25.373904303486427, 25.641889727063145], [15.970058946965699, 17.098746391548367], [32.070953946207965, 29.58292668775776], [21.008658994434107, 26.71935554533437], [23.533977871821612, 20.759917934866944], [19.426409213847684, 16.80296097688458], [23.1886413765842, 20.855202621001688], [58.22269839415908, 51.43169583124266], [89.78930483596196, 90.91765422000655], [24.43060075095525, 22.364376389412907], [58.00693834605711, 55.45580525287736], [55.46312974741875, 61.625966385481036], [9.216964338808147, 5.1250035566184025], [18.4212119614468, 14.068175279699265], [33.6016106704103, 31.1077613628213], [15.62955824880666, 12.893372213559928], [36.36647023003749, 36.15480768427358], [57.14710645443117, 57.18654388894811], [32.842660490673794, 39.622779198392585], [57.08107196690455, 58.078513055401395], [95.28080900842313, 89.1059438290878], [6.842233057329537, 8.169088736934718], [18.14434088607456, 19.70912451868545], [34.094826481803636, 31.368841253114837], [37.66728951973569, 35.415015206183995], [77.54190377253232, 77.39578454932166], [52.96492080038786, 52.50868319684943], [22.63453923833446, 15.070022717471318], [33.79743460475951, 33.39998246347071], [40.07411083738265, 38.73375442000272], [27.030317016778035, 25.37299789931374], [26.778255042920875, 27.38844864044142], [33.90852965514685, 35.9077834768535], [45.44641518045803, 45.32172119889385], [30.733955358821632, 29.30551289775366], [41.504366101298366, 46.210393614641106], [36.203265224759775, 34.29253220960593], [44.76704779174948, 53.801582132809514], [27.150222853555228, 25.754407532871454], [29.957986130610976, 28.93611313683466], [27.50044555154729, 26.044077698836563], [36.84929086058053, 34.70175959550599], [28.770471464709956, 27.28110454010344], [32.115863674927624, 31.21984390004482], [62.328517830232535, 67.71686804702877], [73.21003978307616, 78.76093037589499], [57.3880582816073, 57.67298219243544], [31.447456105343647, 28.69764329085092], [27.94811987241451, 33.613864809191426], [34.093856295727335, 32.69985245382304], [48.84157455613294, 49.58540632071548], [48.16070316851493, 56.45522972911007], [50.74410539511665, 50.89149731175188], [23.036154533912253, 22.303535263890232], [21.372789355118183, 20.60289798727051], [17.554902247109506, 17.44993046728868], [32.01248210991526, 28.6009215752173], [29.992948509984842, 32.1194210278159], [32.544938052844145, 34.352137076934795], [74.26760715059118, 83.60355257438673], [85.10023713850802, 74.21275782787683], [69.01795215144094, 51.21784820061205], [25.067439684905448, 23.50309565880288], [13.36782419715125, 11.553923177938866], [30.182500057172604, 31.595453722246717], [7.932689509082223, 10.510555711199501], [15.67222942634379, 21.084286298973822], [13.142900143178316, 18.068891443984846], [21.209113980447768, 30.33190990887595], [19.866260611042634, 11.43705357620152], [25.569349744223565, 20.80954317246059], [21.439621203761817, 19.56359066244947], [30.052622528446143, 19.57183855119802], [8.252278270050368, 4.166980027251441], [14.189849461112928, 12.577036480956089], [9.962223914276478, 10.70573169374326], [16.506716086285678, 20.99789248508865], [8.903865182623292, 9.209284588371005], [20.829759919094727, 21.16183458709432], [61.29437561915688, 59.24106281853079], [52.60488759724541, 49.13282114422688], [69.67364824363534, 70.64463975731086], [30.220120098641885, 33.180680460555024], [29.711961284526247, 32.315213604659796], [23.13946777620035, 24.7371089447756], [52.03905118383034, 46.088141965437046], [45.16412504393956, 36.3762569612137], [33.549857040486586, 29.66184449190183], [-1.1306637993744317, -2.2268302575289702], [-3.0295302902898698, 0.24138801071953964], [-1.775713981881477, -5.147119471275536], [22.391251896992458, 24.04747807938182], [27.004427277891466, 30.368939379746763], [23.763821430226415, 22.133661320099236], [74.39672409301147, 66.76078589705278], [61.28474601649428, 65.99279140742084], [53.19776537659254, 44.840935505255345], [8.557746398099766, 5.140621126962401]]
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/best_worst_per_point.png'
metrics_dict = {'R2': (True, <function r2_score at 0x7f7a70c10378>), 'root_mean_squared_error': (False, <function root_mean_squared_error at 0x7f7a70c10400>), 'mean_absolute_error': (False, <function mean_absolute_error at 0x7f7a7279b730>), 'rmse_over_stdev': (False, <function rmse_over_stdev at 0x7f7a70c10488>)}
avg_stats = OrderedDict([('R2', (0.728583420179223, 0.029708756915083467)), ('root_mean_squared_error', (50.033697750763096, 5.390383815145658)), ('mean_absolute_error', (33.4870244807647, 1.994334327957242)), ('rmse_over_stdev', (0.5255926774493831, 0.06531233279425615))])
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_best_worst_per_point(y_true, y_pred_list, savepath, metrics_dict, avg_stats, label)
display(Image(filename='best_worst_per_point.png'))
